In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("Example.tsv", sep='\t',header = None).dropna(axis=1)

In [3]:
output =data.iloc[:,0]
inputs =data.iloc[:,1:]

data_A = data[data[0]=="A"]
data_B = data[data[0]=="B"]

inputsA =data_A.iloc[:,1:]
inputsB =data_B.iloc[:,1:]


In [4]:
ProbA = len(inputsA)/len(data)
ProbB = len(inputsB)/len(data)

In [5]:
def getmeanandvariance(data):
    mean = data.sum()/len(data)
    variance = np.sum(np.square(data - mean))/(len(data)-1)
    return mean,variance

In [6]:
def getcontinuousprob(data,meanvariancedata):
    #data: array of x
    #meanvariancedata: data to get mean and variance
    #return probability of x given distribution of meanvariancedata
    mean,variance = getmeanandvariance(meanvariancedata)
    array = np.exp(-np.square(data-mean)/(2*variance))/np.sqrt(2*np.pi*variance)
    return array

In [7]:
inputs["A"] = ProbA
inputs["B"] = ProbB
for i in range(inputsA.shape[1]):
    inputs["A"] = inputs["A"]*getcontinuousprob(inputs.iloc[:,i],inputsA.iloc[:,i])
    inputs["B"] = inputs["B"]*getcontinuousprob(inputs.iloc[:,i],inputsB.iloc[:,i])

In [8]:
inputs.loc[inputs["A"]>=inputs["B"],"prediction"] = "A"
inputs.loc[inputs["A"]<inputs["B"],"prediction"] = "B"


In [9]:
meanA, variA = getmeanandvariance(inputsA)
meanB, variB = getmeanandvariance(inputsB)

In [10]:
outputStr=''
for i in range(inputsA.shape[1]):
    outputStr += str(meanA.to_list()[i]) + "\t"
    outputStr += str(variA.to_list()[i]) + "\t"
outputStr += str(ProbA)
outputStr+="\n"

In [11]:
for i in range(inputsA.shape[1]):
    outputStr += str(meanB.to_list()[i]) + "\t"
    outputStr += str(variB.to_list()[i]) + "\t"
outputStr += str(ProbB)
outputStr+="\n"

In [12]:
outputStr += str((inputs.prediction == output).value_counts().loc[False])

In [13]:
with open('output2.tsv', "w") as outputter:
    outputter.write(outputStr)